<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Christopher Lara
- Nombre de alumno 2:Abiam Tamburrini


### **Link de repositorio de GitHub:** `https://github.com/ChrisLara95/Labs/tree/main`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

#Importamos librerias utiles 😸

In [318]:
# Libreria Core del lab.
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade sklearn
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
!pip install --upgrade graphviz
!pip install --upgrade scipy
import graphviz
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.neighbors import LinearSCV

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

Requirement already up-to-date: numpy in /Users/christopherlara/DS_course/Labs/testenv/lib/python3.8/site-packages (1.21.4)
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: pandas in /Users/christopherlara/DS_course/Labs/testenv/lib/python3.8/site-packages (1.3.4)
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: sklearn in /Users/christopherlara/DS_course/Labs/testenv/lib/python3.8/site-packages (0.0)
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: graphviz in /Users/christopherlara/DS_course/Labs/testenv/lib/python3.8/site-packages (0.18.2)
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: scipy in /Users/christopherlara/DS_course/Labs/testenv/lib/python3.8/site-packages (1.7.2)
You should consider upgrading via the 'pip install --upgrade pip' command.


ImportError: cannot import name 'LinearSCV' from 'sklearn.neighbors' (/Users/christopherlara/DS_course/Labs/testenv/lib/python3.8/site-packages/sklearn/neighbors/__init__.py)

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [205]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [206]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [207]:
seleccionados=df_players['National_Position'].notna()

df_players['Seleccionados']=seleccionados.astype(str)
df_players['Weak_foot']=df_players['Weak_foot'].astype(int)
df_players['Skill_Moves']=df_players['Skill_Moves'].astype(int)


left_foot=(df_players['Preffered_Foot']=='Left')
df_players['Preffered_Foot']=left_foot.astype(int)

df_players.head(5)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionados
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,0,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,True
1,Lionel Messi,Argentina,RW,RW,170,72,1,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,True
2,Neymar,Brazil,LW,LW,174,68,0,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,True
3,Luis Suárez,Uruguay,LS,ST,182,85,0,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,True
4,Manuel Neuer,Germany,GK,GK,193,92,0,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,True


In [208]:
seleccionados.groupby(seleccionados).size()

National_Position
False    16513
True      1075
Name: National_Position, dtype: int64

In [209]:
print('Los no seleccionados son ', (16513)/(16513+1075)*100)
print('Los seleccionados son ', (1075)/(16513+1075)*100)

Los no seleccionados son  93.88787809870365
Los seleccionados son  6.112121901296338


In [210]:
#De los resultados previos observamos que existen 1075 jugadores
#seleccionados, correspondientes a 6.11% del df.
#Mientras que 16513 jugadores no son seleccionados y corresponden al
#93.89% de la muestra, por lo que nuestro df tiene una gran representa_
#tividad de jugadores no seleccionados.
#Dentro de los seleccionados tenemos a los titulares con su respectiva
#posición y a los suplentes como 'sub'

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [211]:
df_players.head(3)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionados
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,0,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,True
1,Lionel Messi,Argentina,RW,RW,170,72,1,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,True
2,Neymar,Brazil,LW,LW,174,68,0,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,True


In [212]:
processing_transform = ColumnTransformer(
    transformers=[
        ('standard_scaler',StandardScaler(),['Height','Weight',
        'Reactions','Finishing','Curve','Penalties','Volleys',
        'Skill_Moves','Weak_foot']),
        ('minmax_scaler',MinMaxScaler(),['Age']),
        ('pt_bc_scaler',PowerTransformer(method='box-cox'),
        ['Ball_Control','Dribbling','Marking','Sliding_Tackle',
        'Standing_Tackle','Aggression','Interceptions','Vision',
        'Composure','Crossing','Short_Pass','Long_Pass','Acceleration',
        'Speed','Stamina','Strength','Balance','Agility','Jumping',
        'Heading','Shot_Power','Long_Shots','Freekick_Accuracy']),
        ('One_hot',OneHotEncoder(),['Work_Rate'])
    ])

first_pipeline = Pipeline([('preprocesado', processing_transform),
    ('tree',DecisionTreeClassifier(criterion="entropy"))])

second_pipeline = Pipeline([('preprocesado', processing_transform),
    ('knn', KNeighborsClassifier(n_neighbors=3))])


#dejamos este tercer pipeline a ejecutar como parte de la tarea pero 
#tuvimos problemas de descargas con el paquete

#third_pipeline = Pipeline([('preprocesado', processing_transform),
#    ('svc_linear', LinearSVC(random_state=0, tol=1e-05))])



In [213]:
#Se escogió StandardScaler() para las variables que inicialmente seguían
#un comportamiento de distriubución normal. En donde nos basamos en los
#gráficos mostrados por el link de ayuda. Al seguir esta distribución,
#solo bastaba con escalar los datos respectivos.

#Se utilizó MinMax() solamente para la edad, por la forma en que esta 
#variable estaba representada en el histograma, además de que no 
#encontramos relevante de que la edad estuviera estrictamente en la 
#escala de las otras variables, ya que no es condición necesaria para
#se seleccionado o no

#para el caso del PowerTransformes, lo utilizamos para los casos en que 
#no seguían un perfil gaussiano, por lo que forzamos a esta condición.
#En especial con Box_Cox, ya que es para cuando los valores son positivos.
#Creemos que las variables que presentaban un perfil de precuencias con 
#dos jorobas, podríamos haber utilizado QuantilTransform, pero no lo 
#hemos visto en clases y tampoco fue propuesto en un comienzo.

#Si bien existían variables categóricas, habían algunas no relevantes 
#para el estudio (como la nacionalidad o club position) y otras que se 
#podían utilizar mediante type(int), por lo cual procedimos en esa 
#dirección. Es por ello que solo utilizamos OneHotEncoder() para la 
#variables indicada en el código.

#Utilizamos los clasificadores mencionados, ya que nos permite predecir
#una variable no numérica. El clasificador de árbol permite una fácil 
#visualización y que adopta criterios para especificar si pertenece o no
#a un conjunto. El caso de kkn, ya que nos permite fácilmente identificar
#si pertenece al grupo de seleccionables o no. De igual manera esperamos
#con el caso de svc linear, donde los planos nos permitan distinguirlo.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [214]:
features = df_players.drop(columns=["Seleccionados"])
labels = df_players.loc[:, "Seleccionados"]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels)

In [215]:
#1-train tree

In [216]:
tree_pipe = first_pipeline.fit(X_train, y_train)

In [217]:
#1-predict tree

In [218]:
y_pred_tree = first_pipeline.predict(X_test)
y_pred_tree

array(['False', 'False', 'False', ..., 'False', 'True', 'False'],
      dtype=object)

In [219]:
#2-train knn

In [220]:
knn_pipe = second_pipeline.fit(X_train, y_train)

In [221]:
#2-predict knn

In [222]:
y_pred_knn = second_pipeline.predict(X_test)
y_pred_knn

array(['False', 'False', 'False', ..., 'False', 'False', 'False'],
      dtype=object)

In [319]:
#3-train SVC

In [320]:
#svc_pipe=third_pipeline.fit(X_train, y_train)

In [321]:
#3-predict SVC

In [322]:
#y_pred_svc = third_pipeline.predict(X_test)

In [223]:
#clasificador tree
print(classification_report(y_test, y_pred_tree))

              precision    recall  f1-score   support

       False       0.95      0.94      0.94      5450
        True       0.18      0.22      0.20       355

    accuracy                           0.89      5805
   macro avg       0.57      0.58      0.57      5805
weighted avg       0.90      0.89      0.90      5805



In [224]:
#clasificador knn
print(classification_report(y_pred_knn, y_test))

              precision    recall  f1-score   support

       False       0.99      0.94      0.97      5701
        True       0.10      0.35      0.16       104

    accuracy                           0.93      5805
   macro avg       0.54      0.65      0.56      5805
weighted avg       0.97      0.93      0.95      5805



In [ ]:
#clasificador svc
print(classification_report(y_pred_svc, y_test))

**Respuesta**:

```
de las matrices obtenidas, podemos notar que en el caso knn la precisión de true aumenta cuando tomamos una menor cantidad de vecinos.

si comparamos knn y tree, observamos que obtenemos un mayor porcentaje de presición de datos falsos que efectivamente son falsos (no convocados) para el caso knn. Sin embargo, si bien para ambos casos es bajo el porcentaje de jugadores seleccionados (True), es más alto para el caso tree. Es decir, que el método de árbol acierta de mejor manera a los jugadores seleccionados, encomparación al método knn.

Mientras que recall nos indica que un comportamiento similar en ambos casos para indicar jugadores no seleccionados, es decir que los encaja de manera correcta. Sin embargo, el knn es mejor para indicar a los casos seleccionados que se predijeron de manera correcta.

Del accuracy podemos desprender que knn fue más exacto al momento de predecir a  los jugadores seleccionados

El número de elementos influye en los resultados, ya que el 96% de los
datos corresponden a jugadores no seleccionados, lo que además coincide
con mejores predicciones para esa tipo de jugadores.
Una de las formas de mejorar, es poder tener información de sobre que 
característica es más importante en cierto tipo de posiciones o no. Así
también con otras características de jugadores. 
También podría ayudar tener una mayor cantidad de datos de jugadores 
seleccionados, incluso, que lo hayan sido pero ya no.
```



## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [279]:
df_players2=df_players.copy(deep=True)
df_players2['Club_Position'] = df_players['Club_Position'].map({
    'ST':'ataque', 'CF':'ataque',
    'RW':'central_ataque','CAM':'central_ataque','LW':'central_ataque',
    'RM':'central', 'CM':'central', 'LM':'central',
    'RWB':'central_defensa', 'CDM':'central_defensa', 'LWB':'central_defensa',
    'RB':'defensa', 'CB':'defensa', 'LB':'defensa',
    'GK':'arquero',
    'Sub':np.nan,'Res':np.nan},na_action=None)
df_players2=df_players2.dropna()

#no creemos relevante la siguiente columna para la prediccion
df_players2=df_players2.drop(['National_Position'], axis=1)

df_players2.head(3)

,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionados
0,Cristiano Ronaldo,Portugal,central_ataque,185,80,0,32,High / Low,4,5,...,95,85,92,93,90,81,76,85,88,True
1,Lionel Messi,Argentina,central_ataque,170,72,1,29,Medium / Medium,4,4,...,68,71,85,95,88,89,90,74,85,True
2,Neymar,Brazil,central_ataque,174,68,0,25,High / Medium,5,5,...,61,62,78,89,77,79,84,81,83,True


In [280]:
df_players2['Club_Position'].value_counts()

defensa            88
arquero            80
central            70
central_ataque     65
ataque             42
central_defensa    21
Name: Club_Position, dtype: int64

In [227]:
#del df obtuvimos que tenemos 80 arqueros, 88 defensas, 21 medios 
#defensivos, 70 medios, 65 medios ofensivos y 42 delanteros.
#al eliminar a los Sub y Res, el df disminuyo enormemente.

In [297]:
fourth_pipeline = Pipeline([('preprocesado', processing_transform),
    ('knn', KNeighborsClassifier(n_neighbors=9))])

In [298]:
features2 = df_players2.drop(columns=["Club_Position"])
labels2 = df_players2.loc[:, "Club_Position"]

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    features2, labels2, test_size=0.33, shuffle=True, stratify=labels2)

In [299]:
knn_pipe2 = fourth_pipeline.fit(X_train2, y_train2)

In [300]:
y_pred_knn2 = fourth_pipeline.predict(X_test2)

In [301]:
print(classification_report(y_pred_knn2, y_test2))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00        26
         ataque       0.79      0.65      0.71        17
        central       0.65      0.47      0.55        32
 central_ataque       0.27      0.55      0.36        11
central_defensa       0.29      1.00      0.44         2
        defensa       0.90      0.79      0.84        33

       accuracy                           0.71       121
      macro avg       0.65      0.74      0.65       121
   weighted avg       0.77      0.71      0.73       121



**Respuesta**:

```
los datos arrojados a continuación fueron realizados para n_neighbours=3, sin embargo, cuando ejecutamos nuevamente, no funcionó.
Después de varias pruebas, logramos con n_n=9, lo que mostró una mejora en los datos test.


del método escogido notamos que el valor del accuracy nos inidica un 0.6, lo que nos indica que predice de manera relativamente exacta. Si 
bien podria ser mejor, está sobre 0.5, por lo cual lo consideramos en 
un rango positivo.
Corrección: para n_n=9 sube a 0.71


de la matriz, el clasificador funciona de manera perfecta para los 
arqueros, lo cual tiene coherencia con las características específicas 
del puesto, que suelen ser distintas a las de otros puesto, los que si
comparten mayores características.

las posiciones más precisas fueron los delanteros y defensas, es decir que predijo una alta cantidad de estos jugadores y que pertenecen a dichas clases. Lo que también es coherente con que las características de estos sueles estár mayormente definidas, salvo en los laterales.

en cuanto a los volantes(central) es donde hay menor cantidad de aciertos con su clase, ya que estas características suelen ser más similares. 
Corrección: al eliminar la columna de posicion en la selección, y aumentar la cantidad de recursividad, este valor aumentó significativamente.

en cuanto a la clasificación de los elementos, los porcentajes disminuyen pero suelen seguir el mismo orden entre los elementos predichos.
Corrección: cuando se ejecutó con n_n=9, los resultados mejoraron considerablemente para los central_defensa correspondientes a dicha clase. Acertanto todos.

Aumentar la cantidad de casos podría ayudar a mejorar nuestros resultados, o separar de manera más especificas algunas posiciones, o incluso realizar una mezcla. Ya que en características de juego podrían 
tener mucho en comun central/central_ataque, laterales (cierto grupo de defensas) y punteros (cierto grupo de delanteros)
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica R^2 predice resultados, tal que el valor obtenido determina la calidad del modelo utilizado, que tiene como ventaja cuenta que es de libre escala. Además es un valor entre -inf y 1, en donde 1 implica un modelo excelente, mientras que un valor negativo significa que el modelo es peor que predecir la media.

In [304]:
df_salarios = pd.read_csv("salarios.csv")
df_salarios = df_salarios.drop(['Unnamed: 0'],axis=1)
df_salarios = df_salarios.rename({'Player': 'Name'}, axis=1)
df_salarios.head(3)

,Name,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0


In [308]:
proyecciones=pd.merge(left=df_players2,
         right=df_salarios,
         on="Name",
         how="inner",
         sort=True)

proyecciones.head(3)

,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionados,Weekly Salary
0,Adama Traoré,Ivory Coast,defensa,170,73,1,27,High / Medium,2,2,...,57,64,52,62,65,34,53,59,True,56731.0
1,Adrien Rabiot,France,central_defensa,188,71,1,21,High / Medium,3,2,...,75,79,66,76,67,57,68,68,True,65029.0
2,Alessandro Schöpf,Austria,central,178,75,0,23,High / High,5,4,...,44,68,72,72,69,61,64,65,True,43345.0


In [335]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression

In [336]:
regressor = LinearRegression()
sueldos_pipe = Pipeline([('preprocesado', processing_transform),
    ('ttr', TransformedTargetRegressor(regressor=regressor))])

In [337]:
features = proyecciones.drop(columns=["Weekly Salary"])
labels = proyecciones.loc[:, "Weekly Salary"]

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    features, labels, test_size=0.33, shuffle=True)


In [352]:
regre_sueldo_pipe = sueldos_pipe.fit(X_train_f, y_train_f)

y_pred_sueldos = sueldos_pipe.predict(X_test_f)

print (sueldos_pipe.score(X_train_f,y_train_f))

0.6908835218739928


**Respuesta**:

```
A partir de los datos de prueba observamos R^2=0.69, por lo cual cumple con lo solicitados.

Una mayor cantidad de datos podría siempre podría ser útil para tener una mayor información. Se podrían también utilizar otro tipo de datos, ya que el sueldo de un futbolista también esta marcado por el impacto de marketing, por lo que podemos considerar una nueva variable asociado a ese punto. Así como también, su duración en la selección o importancia en el equipo.
```


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>